In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds


In [ ]:
!wget https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz

--2021-02-12 12:41:12--  https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
Resolving ai.stanford.edu (ai.stanford.edu)... 171.64.68.10
Connecting to ai.stanford.edu (ai.stanford.edu)|171.64.68.10|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 84125825 (80M) [application/x-gzip]
Saving to: ‘aclImdb_v1.tar.gz’

aclImdb_v1.tar.gz   100%[===================>]  80.23M  22.9MB/s    in 3.5s    

2021-02-12 12:41:16 (22.9 MB/s) - ‘aclImdb_v1.tar.gz’ saved [84125825/84125825]



In [ ]:
!tar -xf aclImdb_v1.tar.gz

In [ ]:
!ls

aclImdb  aclImdb_v1.tar.gz  sample_data


In [ ]:
dataset, info = tfds.load('imdb_reviews/subwords8k', with_info=True, as_supervised=True)

train_dataset, test_dataset = dataset['train'], dataset['test']

encoder = info.features['text'].encoder

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/subwords8k/1.0.0.incompleteXEICA2/imdb_reviews-train.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/subwords8k/1.0.0.incompleteXEICA2/imdb_reviews-test.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/subwords8k/1.0.0.incompleteXEICA2/imdb_reviews-unsupervised.tfrecord


Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/subwords8k/1.0.0. Subsequent calls will reuse this data.


In [ ]:
type(encoder)

tensorflow_datasets.core.deprecated.text.subword_text_encoder.SubwordTextEncoder

In [ ]:
BUFFER_SIZE = 10000
BATCH_SIZE = 64

In [ ]:
padded_shapes = ([None], ())

In [ ]:
train_dataset = train_dataset.shuffle(BUFFER_SIZE).padded_batch(BATCH_SIZE, 
                                                                padded_shapes=padded_shapes)

In [ ]:
test_dataset = test_dataset.padded_batch(BATCH_SIZE, padded_shapes=padded_shapes)

In [ ]:
model = tf.keras.Sequential([tf.keras.layers.Embedding(encoder.vocab_size, 64),
                             tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True)),
                             tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
                             tf.keras.layers.Dense(64, activation= 'relu'),
                             tf.keras.layers.Dropout(0.5),
                             tf.keras.layers.Dense(1,activation="sigmoid")])

In [ ]:
model.compile(loss='binary_crossentropy',
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

history = model.fit(train_dataset, epochs=10, validation_data = test_dataset,
                    validation_steps=30)

def pad_to_size(vec, size):
  zeros = [0]*(size-len(vec))
  vec.extend(zeros)
  return vec

def sample_predict(sentence, pad):
  encoded_sample_pred_text = encoder.encode(sentence)
  if pad:
    encoded_sample_pred_text = pad_to_size(encoded_sample_pred_text, 64)
  encoded_sample_pred_text = tf.cast(encoded_sample_pred_text, tf.float32)
  predictions = model.predict(tf.expand_dims(encoded_sample_pred_text, 0))
  return predictions


sample_text = ('This movie was emotional. The actors were perfect. A must watch')
predictions = sample_predict(sample_text, pad=True)*10

Epoch 1/10
391/391 [==============================] - 82s 182ms/step - loss: 0.6829 - accuracy: 0.5361 - val_loss: 0.4964 - val_accuracy: 0.7599
Epoch 2/10
391/391 [==============================] - 71s 180ms/step - loss: 0.3979 - accuracy: 0.8394 - val_loss: 0.3450 - val_accuracy: 0.8635
Epoch 3/10
391/391 [==============================] - 71s 181ms/step - loss: 0.2785 - accuracy: 0.9026 - val_loss: 0.3676 - val_accuracy: 0.8599
Epoch 4/10
391/391 [==============================] - 70s 179ms/step - loss: 0.2223 - accuracy: 0.9267 - val_loss: 0.3352 - val_accuracy: 0.8687
Epoch 5/10
391/391 [==============================] - 71s 180ms/step - loss: 0.1851 - accuracy: 0.9417 - val_loss: 0.3707 - val_accuracy: 0.8708
Epoch 6/10
391/391 [==============================] - 71s 180ms/step - loss: 0.1497 - accuracy: 0.9576 - val_loss: 0.3782 - val_accuracy: 0.8594
Epoch 7/10
391/391 [==============================] - 71s 180ms/step - loss: 0.1334 - accuracy: 0.9639 - val_loss: 0.4846 - val_ac

In [ ]:
print('Predicted review based on IMDB data%.2f' %predictions)

Predicted review based on IMDB data9.78


In [ ]:
#transfer to django web app for real time consumer feedback

In [ ]:
model.save('MovieClassifier.h5')

In [ ]:
sample_text = ('How wish i can stop falling in love with Nazo Ekezie')
predictions = sample_predict(sample_text, pad=True)*10
print('Predicted review based on IMDB data%.2f' %predictions)

Predicted review based on IMDB data9.62
